# Capstone Project

#### The Battle of Neighborhoods (W1)

Importing libraries

In [6]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import urllib.request
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

Libraries imported.


In [8]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Data downloaded!


"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [9]:
#Tranform the data into a pandas dataframe
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [10]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


### Use geopy library to get the latitude and longitude values of New York City

In [11]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


### Create a map of New York with neighborhoods superimposed on top.

In [12]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [13]:
import folium
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Borough'], manhattan_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

### Foursquare venues

In [14]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [17]:
LIMIT = 500 
radius = 5000 
CLIENT_ID = 'NV1I4KVGFOQCMNNHENCEHFKZX3MZYLOQN5YGN12NUCMMP5JG'
CLIENT_SECRET = 'X3BZBZJOHLGEIDJ3ZAYPHMXC12CQPULXYEBCSCITBK5SAZV5'
VERSION = '20181020'

In [88]:
#https://developer.foursquare.com/docs/resources/categories
#Sushi = 4bf58dd8d48988d153941735
neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
newyork_venues_cr = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d1ca941735')
newyork_venues_cr.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Sam's Pizza,40.879435,-73.905859,Pizza Place
2,Marble Hill,40.876551,-73.91066,Papa John's Pizza,40.878467,-73.902997,Pizza Place
3,Marble Hill,40.876551,-73.91066,Cafeccino Bakery,40.880209,-73.906884,Bagel Shop
4,Marble Hill,40.876551,-73.91066,Supreme Pizza & Pasta,40.881001,-73.908966,Pizza Place


In [89]:
newyork_venues_cr.shape

(1614, 7)

In [90]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [91]:
map_newyork_rest = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(newyork_venues_cr, 'red', map_newyork_rest)

map_newyork_rest

In [92]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0

In [93]:
manhattan_grouped = newyork_venues_cr.groupby('Neighborhood').count()
manhattan_grouped

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,42,42,42,42,42,42
Carnegie Hill,49,49,49,49,49,49
Central Harlem,24,24,24,24,24,24
Chelsea,48,48,48,48,48,48
Chinatown,49,49,49,49,49,49
Civic Center,49,49,49,49,49,49
Clinton,50,50,50,50,50,50
East Harlem,44,44,44,44,44,44
East Village,47,47,47,47,47,47


### Analyze Each Neighborhood

In [94]:
# one hot encoding
manhattan_onehot = pd.get_dummies(newyork_venues_cr[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = newyork_venues_cr['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,American Restaurant,Arcade,Bagel Shop,Bakery,Bar,Boat or Ferry,Burger Joint,Café,Deli / Bodega,Diner,Fast Food Restaurant,Food Truck,Fried Chicken Joint,Gluten-free Restaurant,Gourmet Shop,Italian Restaurant,Kosher Restaurant,Mexican Restaurant,Pizza Place,Sandwich Place,Sports Bar,Turkish Restaurant
0,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,Marble Hill,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [95]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,American Restaurant,Arcade,Bagel Shop,Bakery,Bar,Boat or Ferry,Burger Joint,Café,Deli / Bodega,Diner,Fast Food Restaurant,Food Truck,Fried Chicken Joint,Gluten-free Restaurant,Gourmet Shop,Italian Restaurant,Kosher Restaurant,Mexican Restaurant,Pizza Place,Sandwich Place,Sports Bar,Turkish Restaurant
0,Battery Park City,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.023810,0.047619,0.02381,0.000000,0.904762,0.000000,0.000000,0.00000
1,Carnegie Hill,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.102041,0.00000,0.000000,0.897959,0.000000,0.000000,0.00000
2,Central Harlem,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.041667,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.958333,0.000000,0.000000,0.00000
3,Chelsea,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.020833,0.020833,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.020833,0.00000,0.020833,0.916667,0.000000,0.000000,0.00000
4,Chinatown,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.122449,0.00000,0.000000,0.877551,0.000000,0.000000,0.00000
5,Civic Center,0.000000,0.000000,0.020408,0.020408,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.102041,0.00000,0.000000,0.857143,0.000000,0.000000,0.00000
6,Clinton,0.000000,0.000000,0.000000,0.000000,0.040000,0.00000,0.000000,0.000000,0.000000,0.000000,0.02,0.000000,0.00000,0.000000,0.000000,0.140000,0.00000,0.000000,0.780000,0.020000,0.000000,0.00000
7,East Harlem,0.022727,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.022727,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.954545,0.000000,0.000000,0.00000
8,East Village,0.000000,0.000000,0.000000,0.000000,0.021277,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.085106,0.00000,0.000000,0.893617,0.000000,0.000000,0.00000
9,Financial District,0.000000,0.000000,0.000000,0.000000,0.020408,0.00000,0.000000,0.000000,0.020408,0.000000,0.00,0.020408,0.00000,0.000000,0.020408,0.040816,0.00000,0.000000,0.877551,0.000000,0.000000,0.00000


In [96]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [97]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Pizza Place,Italian Restaurant,Kosher Restaurant,Gourmet Shop,Turkish Restaurant,Deli / Bodega,Arcade,Bagel Shop,Bakery,Bar
1,Carnegie Hill,Pizza Place,Italian Restaurant,Turkish Restaurant,Diner,Arcade,Bagel Shop,Bakery,Bar,Boat or Ferry,Burger Joint
2,Central Harlem,Pizza Place,Deli / Bodega,Turkish Restaurant,Diner,Arcade,Bagel Shop,Bakery,Bar,Boat or Ferry,Burger Joint
3,Chelsea,Pizza Place,Deli / Bodega,Mexican Restaurant,Italian Restaurant,Café,Turkish Restaurant,Arcade,Bagel Shop,Bakery,Bar
4,Chinatown,Pizza Place,Italian Restaurant,Turkish Restaurant,Diner,Arcade,Bagel Shop,Bakery,Bar,Boat or Ferry,Burger Joint


In [98]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 0, 2, 3, 0, 0, 1, 2, 0, 3])

In [99]:
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Pizza Place,Italian Restaurant,Kosher Restaurant,Gourmet Shop,Turkish Restaurant,Deli / Bodega,Arcade,Bagel Shop,Bakery,Bar
1,Carnegie Hill,Pizza Place,Italian Restaurant,Turkish Restaurant,Diner,Arcade,Bagel Shop,Bakery,Bar,Boat or Ferry,Burger Joint
2,Central Harlem,Pizza Place,Deli / Bodega,Turkish Restaurant,Diner,Arcade,Bagel Shop,Bakery,Bar,Boat or Ferry,Burger Joint
3,Chelsea,Pizza Place,Deli / Bodega,Mexican Restaurant,Italian Restaurant,Café,Turkish Restaurant,Arcade,Bagel Shop,Bakery,Bar
4,Chinatown,Pizza Place,Italian Restaurant,Turkish Restaurant,Diner,Arcade,Bagel Shop,Bakery,Bar,Boat or Ferry,Burger Joint


In [100]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,3,Pizza Place,Bagel Shop,Turkish Restaurant,Diner,Arcade,Bakery,Bar,Boat or Ferry,Burger Joint,Café
1,Manhattan,Chinatown,40.715618,-73.994279,0,Pizza Place,Italian Restaurant,Turkish Restaurant,Diner,Arcade,Bagel Shop,Bakery,Bar,Boat or Ferry,Burger Joint
2,Manhattan,Washington Heights,40.851903,-73.936900,2,Pizza Place,Café,Turkish Restaurant,Diner,Arcade,Bagel Shop,Bakery,Bar,Boat or Ferry,Burger Joint
3,Manhattan,Inwood,40.867684,-73.921210,2,Pizza Place,Turkish Restaurant,Diner,Arcade,Bagel Shop,Bakery,Bar,Boat or Ferry,Burger Joint,Café
4,Manhattan,Hamilton Heights,40.823604,-73.949688,2,Pizza Place,Deli / Bodega,Turkish Restaurant,Diner,Arcade,Bagel Shop,Bakery,Bar,Boat or Ferry,Burger Joint


In [101]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [102]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Pizza Place,Italian Restaurant,Turkish Restaurant,Diner,Arcade,Bagel Shop,Bakery,Bar,Boat or Ferry,Burger Joint
8,Upper East Side,Pizza Place,Italian Restaurant,Turkish Restaurant,Diner,Arcade,Bagel Shop,Bakery,Bar,Boat or Ferry,Burger Joint
9,Yorkville,Pizza Place,Italian Restaurant,Turkish Restaurant,Diner,Arcade,Bagel Shop,Bakery,Bar,Boat or Ferry,Burger Joint
10,Lenox Hill,Pizza Place,Italian Restaurant,Turkish Restaurant,Diner,Arcade,Bagel Shop,Bakery,Bar,Boat or Ferry,Burger Joint
11,Roosevelt Island,Pizza Place,Italian Restaurant,Turkish Restaurant,Diner,Arcade,Bagel Shop,Bakery,Bar,Boat or Ferry,Burger Joint
16,Murray Hill,Pizza Place,Italian Restaurant,American Restaurant,Sandwich Place,Deli / Bodega,Arcade,Bagel Shop,Bakery,Bar,Boat or Ferry
19,East Village,Pizza Place,Italian Restaurant,Bar,Turkish Restaurant,Diner,Arcade,Bagel Shop,Bakery,Boat or Ferry,Burger Joint
24,West Village,Pizza Place,Italian Restaurant,Gluten-free Restaurant,Café,Turkish Restaurant,Deli / Bodega,Arcade,Bagel Shop,Bakery,Bar
30,Carnegie Hill,Pizza Place,Italian Restaurant,Turkish Restaurant,Diner,Arcade,Bagel Shop,Bakery,Bar,Boat or Ferry,Burger Joint
32,Civic Center,Pizza Place,Italian Restaurant,Bagel Shop,Bakery,Turkish Restaurant,Diner,Arcade,Bar,Boat or Ferry,Burger Joint


In [103]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Upper West Side,Pizza Place,Italian Restaurant,Turkish Restaurant,Bakery,American Restaurant,Kosher Restaurant,Café,Arcade,Bagel Shop,Sandwich Place
14,Clinton,Pizza Place,Italian Restaurant,Bar,Fast Food Restaurant,Sandwich Place,Deli / Bodega,Arcade,Bagel Shop,Bakery,Boat or Ferry
15,Midtown,Pizza Place,Italian Restaurant,Sandwich Place,American Restaurant,Deli / Bodega,Arcade,Bagel Shop,Bakery,Bar,Boat or Ferry
18,Greenwich Village,Pizza Place,Italian Restaurant,Sandwich Place,Bagel Shop,Bakery,Turkish Restaurant,Deli / Bodega,Arcade,Bar,Boat or Ferry
21,Tribeca,Pizza Place,Italian Restaurant,Bagel Shop,Bakery,Food Truck,Sports Bar,Turkish Restaurant,Deli / Bodega,Arcade,Bar
22,Little Italy,Pizza Place,Italian Restaurant,Bagel Shop,Bakery,Turkish Restaurant,Diner,Arcade,Bar,Boat or Ferry,Burger Joint
23,Soho,Pizza Place,Italian Restaurant,Bagel Shop,Bakery,Turkish Restaurant,Diner,Arcade,Bar,Boat or Ferry,Burger Joint
31,Noho,Pizza Place,Italian Restaurant,Turkish Restaurant,Diner,Arcade,Bagel Shop,Bakery,Bar,Boat or Ferry,Burger Joint
33,Midtown South,Pizza Place,Italian Restaurant,Mexican Restaurant,Turkish Restaurant,Deli / Bodega,Arcade,Bagel Shop,Bakery,Bar,Boat or Ferry


In [104]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Washington Heights,Pizza Place,Café,Turkish Restaurant,Diner,Arcade,Bagel Shop,Bakery,Bar,Boat or Ferry,Burger Joint
3,Inwood,Pizza Place,Turkish Restaurant,Diner,Arcade,Bagel Shop,Bakery,Bar,Boat or Ferry,Burger Joint,Café
4,Hamilton Heights,Pizza Place,Deli / Bodega,Turkish Restaurant,Diner,Arcade,Bagel Shop,Bakery,Bar,Boat or Ferry,Burger Joint
6,Central Harlem,Pizza Place,Deli / Bodega,Turkish Restaurant,Diner,Arcade,Bagel Shop,Bakery,Bar,Boat or Ferry,Burger Joint
7,East Harlem,Pizza Place,American Restaurant,Deli / Bodega,Diner,Arcade,Bagel Shop,Bakery,Bar,Boat or Ferry,Burger Joint
20,Lower East Side,Pizza Place,Italian Restaurant,Turkish Restaurant,Diner,Arcade,Bagel Shop,Bakery,Bar,Boat or Ferry,Burger Joint
27,Gramercy,Pizza Place,Italian Restaurant,Bar,Turkish Restaurant,Diner,Arcade,Bagel Shop,Bakery,Boat or Ferry,Burger Joint
34,Sutton Place,Pizza Place,Sandwich Place,Italian Restaurant,Turkish Restaurant,Deli / Bodega,Arcade,Bagel Shop,Bakery,Bar,Boat or Ferry
35,Turtle Bay,Pizza Place,Sandwich Place,Italian Restaurant,Turkish Restaurant,Deli / Bodega,Arcade,Bagel Shop,Bakery,Bar,Boat or Ferry
36,Tudor City,Pizza Place,Italian Restaurant,Turkish Restaurant,Diner,Arcade,Bagel Shop,Bakery,Bar,Boat or Ferry,Burger Joint


In [105]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Pizza Place,Bagel Shop,Turkish Restaurant,Diner,Arcade,Bakery,Bar,Boat or Ferry,Burger Joint,Café
13,Lincoln Square,Pizza Place,Deli / Bodega,Bakery,Italian Restaurant,Burger Joint,Turkish Restaurant,Diner,Arcade,Bagel Shop,Bar
17,Chelsea,Pizza Place,Deli / Bodega,Mexican Restaurant,Italian Restaurant,Café,Turkish Restaurant,Arcade,Bagel Shop,Bakery,Bar
25,Manhattan Valley,Pizza Place,Deli / Bodega,Italian Restaurant,Fried Chicken Joint,Turkish Restaurant,Arcade,Bagel Shop,Bakery,Bar,Boat or Ferry
28,Battery Park City,Pizza Place,Italian Restaurant,Kosher Restaurant,Gourmet Shop,Turkish Restaurant,Deli / Bodega,Arcade,Bagel Shop,Bakery,Bar
29,Financial District,Pizza Place,Italian Restaurant,Deli / Bodega,Gourmet Shop,Bar,Food Truck,Turkish Restaurant,Arcade,Bagel Shop,Bakery
37,Stuyvesant Town,Pizza Place,Italian Restaurant,Bar,Boat or Ferry,Turkish Restaurant,Diner,Arcade,Bagel Shop,Bakery,Burger Joint
38,Flatiron,Pizza Place,Italian Restaurant,Sandwich Place,Mexican Restaurant,Café,Turkish Restaurant,Deli / Bodega,Arcade,Bagel Shop,Bakery


In [106]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Manhattanville,Pizza Place,Diner,Arcade,Italian Restaurant,Turkish Restaurant,Bagel Shop,Bakery,Bar,Boat or Ferry,Burger Joint
26,Morningside Heights,Pizza Place,Italian Restaurant,Diner,Arcade,Deli / Bodega,Turkish Restaurant,Bagel Shop,Bakery,Bar,Boat or Ferry
